# ECL

## Part 1 - read dataset

In [1]:
import json
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Read in the dataset.
dataset = pd.read_csv('ECL.csv', index_col=0)

In [3]:
print(len(dataset))

170139


In [206]:
# Get the subset of labelled 10Ks
prediction_subset = dataset.loc[(dataset['can_label'] == True) & (dataset['qualified'] == 'Yes')].reset_index(drop=True)
prediction_subset['cik'] = prediction_subset['cik'].astype(int)

In [207]:
qualified_possible_values = dataset.loc[(dataset['can_label'] == False)]
print(len(qualified_possible_values))

0


In [208]:
prediction_subset.drop(columns=['can_label'], inplace=True)

In [209]:
print('Number of labelled 10Ks:', len(prediction_subset))

Number of labelled 10Ks: 84652


**-> Dataset is ready for further processing**

In [237]:
# getting unique companies by cik
unique_companies = prediction_subset.groupby('cik').agg(company=('company', 'last')).reset_index()
print(f"Number of unique companies: {len(unique_companies)}")
print(unique_companies.head(10))

Number of unique companies: 9143
    cik                           company
0  1750                          AAR CORP
1  1800               ABBOTT LABORATORIES
2  2024                 ACE HARDWARE CORP
3  2034                        ACETO CORP
4  2135  AFFILIATED COMPUTER SERVICES INC
5  2178    ADAMS RESOURCES & ENERGY, INC.
6  2488        ADVANCED MICRO DEVICES INC
7  2491          BALLY TECHNOLOGIES, INC.
8  2601                      AEROFLEX INC
9  2852                         AGWAY INC


## Part 2 - looking into data

In [211]:
companies_with_last_filing_date = prediction_subset.groupby('cik').agg(
    company=('company', 'last'),
    filing_date=('filing_date', 'last')
).reset_index()

print(f"Number of companies with filing date: {len(companies_with_last_filing_date)}")
print(companies_with_last_filing_date.head(10))

Number of companies with filing date: 9143
    cik                           company filing_date
0  1750                          AAR CORP  2021-07-21
1  1800               ABBOTT LABORATORIES  2021-02-19
2  2024                 ACE HARDWARE CORP  2003-03-24
3  2034                        ACETO CORP  2018-09-28
4  2135  AFFILIATED COMPUTER SERVICES INC  2009-08-27
5  2178    ADAMS RESOURCES & ENERGY, INC.  2020-03-06
6  2488        ADVANCED MICRO DEVICES INC  2021-01-29
7  2491          BALLY TECHNOLOGIES, INC.  2014-08-29
8  2601                      AEROFLEX INC  2006-09-13
9  2852                         AGWAY INC  2002-09-30


In [213]:
companies_with_last_filing_date_after_2021 = companies_with_last_filing_date[companies_with_last_filing_date['filing_date'] > '2021-01-01']
print(len(companies_with_last_filing_date_after_2021))

3086


In [214]:
companies_with_last_filing_date_before_2010 = companies_with_last_filing_date[companies_with_last_filing_date['filing_date'] < '2010-01-01']
print(len(companies_with_last_filing_date_before_2010))

3920


Around third of companies have last filing date greater than 2021 - the last year included within the dataset that could be labelled (due to unsupported Florida-UCLA-LoPucki database)

Almost half has last filing before 2010

## Part 3 - looking for tickers using gurufocus.com

I need to get tickers and stockid for companies - to further download financial data from gurufocus.com (or look for different data provider). Also I focus on NYSE and NASDAQ exchanges.

In [4]:
import requests
import time
import re
from tqdm import tqdm
import datetime

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'authority': 'www.google.com',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9',
    'cache-control': 'max-age=0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
}

In [5]:
pattern = re.compile(r"[\\/.,\"]") 

def clear_company_name(company_name):
    company_name = company_name.lower()
    company_name = re.sub(r"[\\/(].*", '', company_name)
    company_name = pattern.sub('', company_name)
    company_name = company_name.strip()
    company_name = company_name.replace(' ', '+')
    return company_name

In [7]:
print(clear_company_name("ALEXANDER & ALEXANDER SERVICES INC"))

alexander+&+alexander+services+inc


In [45]:
tqdm.pandas()

found_companies_count = 0

companies_with_not_found_tickers_list = []
companies_with_not_found_tickers_with_exception = []

pbar = tqdm(unique_companies.values.tolist())

exceptions_list = []

for cik, company in pbar:

    cleared_company_name = clear_company_name(company)
    url = f'https://www.gurufocus.com/reader/_api/_search?v=1.4.13&text={cleared_company_name}'
    
    try:
        
        try:
            response = requests.get(url, headers=headers)
            response_json = response.json()
        except Exception as ex:
            companies_with_not_found_tickers_with_exception.append({'cik': cik, 'company': company})
            exceptions_list.append(ex)
            time.sleep(0.7)
            continue

        time.sleep(0.2)
    
        if len(response_json) == 0:
            has_company_name_changed = False
            if '+inc' in cleared_company_name:
                cleared_company_name = cleared_company_name.replace('+inc', '')
                has_company_name_changed = True
            if '+co' in cleared_company_name:
                # may match 'co' in any word
                cleared_company_name = cleared_company_name.replace('+co', '+company')
                has_company_name_changed = True
            if '+llc' in cleared_company_name:
                cleared_company_name = cleared_company_name.replace('+llc', '')
                has_company_name_changed = True
            cleared_company_name = cleared_company_name.strip()
            
            if has_company_name_changed:
                url = f'https://www.gurufocus.com/reader/_api/_search?v=1.4.13&text={cleared_company_name}'
                try:
                    response = requests.get(url, headers=headers)
                    response_json = response.json()
                except Exception as ex:
                    companies_with_not_found_tickers_with_exception.append({'cik': cik, 'company': company})
                    exceptions_list.append(ex)
                    time.sleep(0.7)
                    continue
                
            if len(response_json) == 0:
                companies_with_not_found_tickers_list.append({'cik': cik, 'company': company})
                continue
        
        is_company_found = False
        matched_type_counter = 0
        for entry in response_json:
            if entry['type'] not in ('stock', 'delisted'):
                if matched_type_counter > 0:
                    break
                continue
                
            exchange = entry['data']['exchange']
            if exchange in ('NYSE', 'NAS', 'DELISTED'):
                ticker = entry['data']['symbol']
                stockid = entry['data']['stockid']
                # gurufocus_company_name = entry['data']['company']
                
                if matched_type_counter == 0:
                    prediction_subset.loc[prediction_subset['cik'] == cik, 'ticker'] = ticker
                    prediction_subset.loc[prediction_subset['cik'] == cik, 'exchange'] = exchange
                    prediction_subset.loc[prediction_subset['cik'] == cik, 'gurufocus-stockid'] = stockid
                    # prediction_subset.loc[prediction_subset['cik'] == cik, 'gurufocus-company-name'] = gurufocus_company_name
                    
                    
                    is_company_found = True
                    found_companies_count += 1
                    pbar.set_description(f"Found companies count: {found_companies_count}")
                    matched_type_counter += 1
                    
                elif matched_type_counter == 1:
                    prediction_subset.loc[prediction_subset['cik'] == cik, 'second-match-ticker'] = ticker
                    prediction_subset.loc[prediction_subset['cik'] == cik, 'second-match-exchange'] = exchange
                    prediction_subset.loc[prediction_subset['cik'] == cik, 'second-match-gurufocus-stockid'] = stockid
                    # prediction_subset.loc[prediction_subset['cik'] == cik, 'second-match-gurufocus-company-name'] = gurufocus_company_name
                else:
                    break
                
        if not is_company_found:
            companies_with_not_found_tickers_list.append({'cik': cik, 'company': company})
            
    except Exception as ex:
        companies_with_not_found_tickers_with_exception.append({'cik': cik, 'company': company})
        exceptions_list.append(ex)
        time.sleep(0.7)
        
companies_with_not_found_tickers_df = pd.DataFrame(companies_with_not_found_tickers_list)
companies_with_not_found_tickers_with_exception_df = pd.DataFrame(companies_with_not_found_tickers_with_exception)

Found companies count: 5774: 100%|██████████| 9143/9143 [3:14:41<00:00,  1.28s/it]  


In [46]:
print(f"Number of companies with found tickers: {found_companies_count}")
print(f"Number of companies with not found tickers: {len(companies_with_not_found_tickers_list)}")
print(f"Number of companies with exceptions: {len(companies_with_not_found_tickers_with_exception_df)}")

Number of companies with found tickers: 5774
Number of companies with not found tickers: 3369
Number of companies with exceptions: 0


In [47]:
datetime_now = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

prediction_subset.to_csv(f'ECL_with_ticker_{datetime_now}.csv')
companies_with_not_found_tickers_df.to_csv(f'ECL_companies_with_not_found_tickers_{datetime_now}.csv')
companies_with_not_found_tickers_with_exception_df.to_csv(f'ECL_companies_with_not_found_tickers_with_exception_{datetime_now}.csv')

In [ ]:
# move ticker and exchange to the front of the dataset
# prediction_subset = prediction_subset[['ticker', 'company', 'exchange', 'label', 'filename', 'bankruptcy_prediction_split', 'filing_date']]

## Download data from gurufocus.com

#### Download data for companies with one ticker (then for companies with two tickers)

In [8]:
prediction_subset = pd.read_csv('ECL_with_ticker_2024-04-14_15-20-33.csv', index_col=0)

In [9]:
prediction_subset_with_gurufocus_data = prediction_subset.groupby('cik').agg(
    company=('company', 'last'),
    ticker=('ticker', 'last'),
    second_match_ticker=('second-match-ticker', 'last'),
    gurufocus_stockid=('gurufocus-stockid', 'last'),
    second_match_gurufocus_stockid=('second-match-gurufocus-stockid', 'last')
).reset_index()

In [19]:
# get companies with one ticker
companies_with_one_ticker = prediction_subset_with_gurufocus_data[prediction_subset_with_gurufocus_data['ticker'].notnull() & prediction_subset_with_gurufocus_data['second_match_ticker'].isnull()]
print(len(companies_with_one_ticker))
print(companies_with_one_ticker.head(10))

5115
     cik                            company ticker second_match_ticker  \
0   1750                           AAR CORP    AIR                None   
1   1800                ABBOTT LABORATORIES    ABT                None   
6   2488         ADVANCED MICRO DEVICES INC    AMD                None   
7   2491           BALLY TECHNOLOGIES, INC.    BYI                None   
8   2601                       AEROFLEX INC   ARXX                None   
10  2969  AIR PRODUCTS & CHEMICALS INC /DE/    APD                None   
16  3197            CECO ENVIRONMENTAL CORP   CECO                None   
19  3370          IKON OFFICE SOLUTIONS INC    IKN                None   
21  3453                       MATSON, INC.   MATX                None   
23  3545                        ALICO, INC.   ALCO                None   

   gurufocus_stockid second_match_gurufocus_stockid  
0             US06AR                           None  
1             US066X                           None  
6             US02

In [267]:
companies_with_one_ticker_grouped_by_cik = companies_with_one_ticker.groupby('cik').agg(
    ticker=('ticker', 'last'),
    gurufocus_stockid=('gurufocus_stockid', 'last')
).reset_index()
print(len(companies_with_one_ticker_grouped_by_cik))
print(companies_with_one_ticker_grouped_by_cik.head(10))

5115
    cik ticker gurufocus_stockid
0  1750    AIR            US06AR
1  1800    ABT            US066X
2  2488    AMD            US022E
3  2491    BYI            US06R5
4  2601   ARXX            US026H
5  2969    APD            US06DU
6  3197   CECO            US02K6
7  3370    IKN            US07ZK
8  3453   MATX            US08CS
9  3545   ALCO            US0212


In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from tqdm import tqdm
from bs4 import BeautifulSoup

table_ids = [
    'financial_table_per_share_data',
    'financial_table_ratios',
    'financial_table_income_statement',
    'financial_table_balance_sheet',
    'financial_table_cashflow_statement',
    'financial_table_valuation_ratios',
    'financial_table_valuation_and_quality'
]

options = webdriver.EdgeOptions()

login_url = 'https://www.gurufocus.com/login'
payload = {
    "username": "darekkruszel15@gmail.com",
    "password": "=OcUZ*5&|{+l7-lGy:QMe4vHyF4X'~"
}

def process_df(df):
    df.drop(df.columns[1], axis=1, inplace=True) 
    return df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [278]:
# login
driver = webdriver.Edge(options=options)
driver.get(login_url)

username = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, 'username')))
password = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, 'password')))
username.clear()
password.clear()
username.send_keys(payload['username'])
password.send_keys(payload['password'])
password.send_keys(Keys.RETURN)

WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.LINK_TEXT, 'Articles')))
time.sleep(1)


not_found_financial_data_with_exception = []
not_found_financial_data_with_webdriver_timeout_exception = []
success_count = 0

pbar = tqdm(companies_with_one_ticker_grouped_by_cik.values.tolist())
try:
    for cik, ticker, stockid in pbar:
        
        url = f'https://www.gurufocus.com/stock/{stockid}/financials'
        
        try:
            driver.get(url)
            time.sleep(0.7)
    
            try:
                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.ID, 'financial_table_per_share_data')))
                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.ID, 'financial_table_ratios')))
                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.ID, 'financial_table_income_statement')))
                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.ID, 'financial_table_balance_sheet')))
                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.ID, 'financial_table_cashflow_statement')))
                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.ID, 'financial_table_valuation_ratios')))
                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.ID, 'financial_table_valuation_and_quality')))
            except Exception as ex:
                not_found_financial_data_with_webdriver_timeout_exception.append([cik, stockid, ticker, str(ex)])
                continue
    
            page_source = driver.page_source
            with open(f'gurufocus-immediate-response-for-stockid/page_source-{cik}-{stockid}_{ticker}.txt', 'w', encoding='utf-8') as f:
                f.write(page_source)
            
            soup = BeautifulSoup(page_source, 'html.parser')
        
            gurufocus_company_name = soup.find('div', {'id': 'stock-header'}).find('div').text
    
            prediction_subset.loc[prediction_subset['cik'] == cik, 'gurufocus-company-name'] = gurufocus_company_name
        
            tables = soup.find_all('table')
            
            page_tables_ids = []
            for table in tables:
                try:
                    table_id = table.get('id')
                    if table_id:
                        page_tables_ids.append(table_id)
                except:
                    pass
            
            if all(table_id not in page_tables_ids for table_id in table_ids):
                pass
            else:
                merged_df = pd.DataFrame()
                for table_id in table_ids:
                    table = soup.find(id=table_id)
                    df = pd.read_html(str(table), skiprows=1, header=0)[0]
                    df = process_df(df)
                    merged_df = pd.concat([merged_df, df])
                    
                merged_df.reset_index(drop=True, inplace=True)
                merged_df.to_csv(f'financial_data/{cik}-{stockid}_{ticker}.csv')
                
                success_count += 1
                pbar.set_description(f"Success: {success_count}")
    
        except Exception as ex:
            not_found_financial_data_with_exception.append([cik, stockid, ticker, str(ex)])

finally:
    driver.close()
    prediction_subset.to_csv(f'ECL_with_ticker_{datetime_now}.csv')

    with open('not_found_financial_data_with_exception.json', 'w') as file:
        json.dump(list(not_found_financial_data_with_exception), file)
        
    with open('not_found_financial_data_with_webdriver_timeout_exception.json', 'w') as file:
        json.dump(list(not_found_financial_data_with_webdriver_timeout_exception), file)

Success: 5071: 100%|██████████| 5115/5115 [5:47:17<00:00,  4.07s/it]   


In [ ]:
print(f"Number of companies with not found financial data with exception: {len(not_found_financial_data_with_exception)}")
print(f"Number of companies with not found financial data with webdriver timeout exception: {len(not_found_financial_data_with_webdriver_timeout_exception)}")

#### Get companies with two tickers

In [11]:
# get companies with two tickers
companies_with_two_tickers = prediction_subset_with_gurufocus_data[prediction_subset_with_gurufocus_data['second_match_ticker'].notnull()]
print(f"Number of companies with two tickers: {len(companies_with_two_tickers)}")
print(companies_with_two_tickers.head(10))

Number of companies with two tickers: 659
     cik                             company     ticker second_match_ticker  \
15  3153                    ALABAMA POWER CO  ALPpQ.PFD         ALPROCL.PFD   
18  3333                 ALBERTSONS INC /DE/        ACI                 ABS   
20  3449  ALEXANDER & ALEXANDER SERVICES INC       ALEX                 AAL   
35  4281               HOWMET AEROSPACE INC.        HWM         ARNCPRB.PFD   
39  4447                           HESS CORP        HES           HESpA.PFD   
56  5272  AMERICAN INTERNATIONAL GROUP, INC.        AIG               AIGWS   
61  5611                            FINA INC        TFC                WYPT   
64  5907                           AT&T CORP          T                  AT   
91  7332              SOUTHWESTERN ENERGY CO        SWN            SWNC.PFD   
97  7789                ASSOCIATED BANC-CORP        ASB           ASBpC.PFD   

   gurufocus_stockid second_match_gurufocus_stockid  
15            US0TUZ              

In [19]:
companies_first_tickers = companies_with_two_tickers[['cik', 'company', 'ticker', 'gurufocus_stockid']]
companies_first_tickers['second_match'] = False

companies_second_tickers = companies_with_two_tickers[['cik', 'company', 'second_match_ticker', 'second_match_gurufocus_stockid']]
companies_second_tickers.columns = ['cik', 'company', 'ticker', 'gurufocus_stockid']
companies_second_tickers['second_match'] = True

companies_with_two_tickers_flatten = pd.concat([companies_first_tickers, companies_second_tickers])

print(len(companies_with_two_tickers_flatten))
print(companies_with_two_tickers_flatten.head(10))

1318
     cik                             company     ticker gurufocus_stockid  \
15  3153                    ALABAMA POWER CO  ALPpQ.PFD            US0TUZ   
18  3333                 ALBERTSONS INC /DE/        ACI            US280P   
20  3449  ALEXANDER & ALEXANDER SERVICES INC       ALEX            US06BL   
35  4281               HOWMET AEROSPACE INC.        HWM            US0SWN   
39  4447                           HESS CORP        HES            US07TL   
56  5272  AMERICAN INTERNATIONAL GROUP, INC.        AIG            US06AN   
61  5611                            FINA INC        TFC            US06JN   
64  5907                           AT&T CORP          T            US09O4   
91  7332              SOUTHWESTERN ENERGY CO        SWN            US09NC   
97  7789                ASSOCIATED BANC-CORP        ASB            US06FE   

    second_match  
15         False  
18         False  
20         False  
35         False  
39         False  
56         False  
61         Fal

In [18]:
# sort companies_with_two_tickers_flatten by company name
companies_with_two_tickers_flatten = companies_with_two_tickers_flatten.sort_values(by='ticker')
print(companies_with_two_tickers_flatten.tail(10))

          cik                         company ticker gurufocus_stockid  \
6072  1094738            M&F BANCORP INC /NC/   ZOLL            US064K   
435     36270                   M&T BANK CORP   ZOLL            US064K   
1989   736473                     F&M BANCORP  ZZDEC            US0SNV   
2815   830143              D&N FINANCIAL CORP  ZZDEC            US0SNV   
4776  1011737          D&E COMMUNICATIONS INC  ZZDEC            US0SNV   
3547   888914  D & K HEALTHCARE RESOURCES INC  ZZDEC            US0SNV   
8351  1581091           RE/MAX HOLDINGS, INC.  ZZDEC            US0SNV   
2011   740806                   F&M BANK CORP  ZZDEC            US0SNV   
412     34125               F&M NATIONAL CORP  ZZDEC            US0SNV   
2387   793049          F&M BANCORPORATION INC  ZZDEC            US0SNV   

      second_match  
6072          True  
435           True  
1989          True  
2815          True  
4776          True  
3547          True  
8351          True  
2011          Tru

In [30]:
# login
driver = webdriver.Edge(options=options)
driver.get(login_url)

username = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, 'username')))
password = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.NAME, 'password')))
username.clear()
password.clear()
username.send_keys(payload['username'])
password.send_keys(payload['password'])
password.send_keys(Keys.RETURN)

WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.LINK_TEXT, 'Articles')))
time.sleep(1)


not_found_financial_data_with_exception = []
not_found_financial_data_with_webdriver_timeout_exception = []
success_count = 0

pbar = tqdm(companies_with_two_tickers_flatten.values.tolist())
try:
    for cik, company, ticker, stockid, _, _ in pbar:
        
        url = f'https://www.gurufocus.com/stock/{stockid}/financials'
        
        try:
            driver.get(url)
            time.sleep(0.6)
    
            try:
                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.ID, 'financial_table_per_share_data')))
                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.ID, 'financial_table_ratios')))
                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.ID, 'financial_table_income_statement')))
                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.ID, 'financial_table_balance_sheet')))
                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.ID, 'financial_table_cashflow_statement')))
                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.ID, 'financial_table_valuation_ratios')))
                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.ID, 'financial_table_valuation_and_quality')))
            except Exception as ex:
                not_found_financial_data_with_webdriver_timeout_exception.append([cik, stockid, ticker, str(ex)])
                continue
    
            page_source = driver.page_source
            with open(f'gurufocus-immediate-response-for-stockid-two-tickers-case/page_source-{cik}-{stockid}_{ticker}.txt', 'w', encoding='utf-8') as f:
                f.write(page_source)
            
            soup = BeautifulSoup(page_source, 'html.parser')
        
            gurufocus_company_name = soup.find('div', {'id': 'stock-header'}).find('div').text
    
            mask = (companies_with_two_tickers_flatten['cik'] == cik) & (companies_with_two_tickers_flatten['gurufocus_stockid'] == stockid)
            companies_with_two_tickers_flatten.loc[mask, 'gurufocus-company-name'] = gurufocus_company_name 

            tables = soup.find_all('table')
            
            page_tables_ids = []
            for table in tables:
                try:
                    table_id = table.get('id')
                    if table_id:
                        page_tables_ids.append(table_id)
                except:
                    pass
            
            if all(table_id not in page_tables_ids for table_id in table_ids):
                pass
            else:
                merged_df = pd.DataFrame()
                for table_id in table_ids:
                    table = soup.find(id=table_id)
                    df = pd.read_html(str(table), skiprows=1, header=0)[0]
                    df = process_df(df)
                    merged_df = pd.concat([merged_df, df])
                    
                merged_df.reset_index(drop=True, inplace=True)
                merged_df.to_csv(f'financial_data_two_tickers_case/{cik}-{stockid}_{ticker}.csv')
                
                success_count += 1
                pbar.set_description(f"Success: {success_count}")
    
        except Exception as ex:
            not_found_financial_data_with_exception.append([cik, stockid, ticker, str(ex)])

finally:
    driver.close()
    
    datetime_now = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    
    companies_with_two_tickers_flatten.to_csv(f'companies_with_two_tickers_flatten_{datetime_now}.csv')

    with open(f'not_found_financial_data_with_exception_two_ticker_case_{datetime_now}.json', 'w') as file:
        json.dump(list(not_found_financial_data_with_exception), file)
        
    with open(f'not_found_financial_data_with_webdriver_timeout_exception_two_ticker_case_{datetime_now}.json', 'w') as file:
        json.dump(list(not_found_financial_data_with_webdriver_timeout_exception), file)


  0%|          | 0/1318 [00:45<?, ?it/s]

Success: 1242: 100%|██████████| 1318/1318 [1:58:50<00:00,  5.41s/it]


# start here again - try to download 76 again

In [31]:
print(f"Number of companies with not found financial data with exception: {len(not_found_financial_data_with_exception)}")
print(f"Number of companies with not found financial data with webdriver timeout exception: {len(not_found_financial_data_with_webdriver_timeout_exception)}")

Number of companies with not found financial data with exception: 0
Number of companies with not found financial data with webdriver timeout exception: 76


In [7]:
import os

directory = 'financial_data'

found_data_tickers = []

for filename in os.listdir(directory):
    if filename.endswith('.csv'):  # Ensure it's a CSV file
        found_data_tickers.append(filename.replace('.csv', ''))

In [10]:
unique_companies_tickers = prediction_subset.groupby('cik').agg(ticker=('ticker', 'last')).reset_index()
number_of_companies_with_not_found_tickers = unique_companies_tickers['ticker'].isnull().sum()
print(f"Number of companies with not found tickers: {number_of_companies_with_not_found_tickers}")

Number of companies with not found tickers: 9099


### Process downloaded data

In [33]:
companies_with_two_tickers_flatten = pd.read_csv('companies_with_two_tickers_flatten_2024-04-16_22-22-08.csv', index_col=0)

In [34]:
print(companies_with_two_tickers_flatten.head(10))

     cik                             company     ticker gurufocus_stockid  \
15  3153                    ALABAMA POWER CO  ALPpQ.PFD            US0TUZ   
18  3333                 ALBERTSONS INC /DE/        ACI            US280P   
20  3449  ALEXANDER & ALEXANDER SERVICES INC       ALEX            US06BL   
35  4281               HOWMET AEROSPACE INC.        HWM            US0SWN   
39  4447                           HESS CORP        HES            US07TL   
56  5272  AMERICAN INTERNATIONAL GROUP, INC.        AIG            US06AN   
61  5611                            FINA INC        TFC            US06JN   
64  5907                           AT&T CORP          T            US09O4   
91  7332              SOUTHWESTERN ENERGY CO        SWN            US09NC   
97  7789                ASSOCIATED BANC-CORP        ASB            US06FE   

    second_match                             gurufocus-company-name  
15         False  \n    Alabama Power Co  (Delisted)\n    (NYSE:...  
18         F

In [41]:
def process_gurufocus_company_name_column(col):
    col = col.str.strip()
    col = col.astype(str)
    col = col.apply(lambda x: x.split('\n')[0])
    return col

In [42]:
companies_with_two_tickers_flatten['gurufocus-company-name'] = process_gurufocus_company_name_column(companies_with_two_tickers_flatten['gurufocus-company-name'])

print(companies_with_two_tickers_flatten.head(10))

       cik                               company     ticker gurufocus_stockid  \
15    3153                      ALABAMA POWER CO  ALPpQ.PFD            US0TUZ   
18    3333                   ALBERTSONS INC /DE/        ACI            US280P   
20    3449    ALEXANDER & ALEXANDER SERVICES INC       ALEX            US06BL   
35    4281                 HOWMET AEROSPACE INC.        HWM            US0SWN   
39    4447                             HESS CORP        HES            US07TL   
56    5272    AMERICAN INTERNATIONAL GROUP, INC.        AIG            US06AN   
61    5611                              FINA INC        TFC            US06JN   
64    5907                             AT&T CORP          T            US09O4   
91    7332                SOUTHWESTERN ENERGY CO        SWN            US09NC   
97    7789                  ASSOCIATED BANC-CORP        ASB            US06FE   
147  12208            BIO-RAD LABORATORIES, INC.        BIO            US06MF   
148  12355                  

# Part 4

Checking tickers for all companies with edgar search

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [5]:
partially_filled_dataset = pd.read_csv('ECL_with_ticker_2024-04-12_19-10-45.csv', index_col=0)

In [7]:
print(partially_filled_dataset.count())

61973


In [3]:
print(partially_filled_dataset.head())

    cik   company period_of_report   gvkey    datadate  \
0  1750  AAR CORP       1994-05-31  1004.0  31/05/1994   
1  1750  AAR CORP       1995-05-31  1004.0  31/05/1995   
2  1750  AAR CORP       1996-05-31  1004.0  31/05/1996   
3  1750  AAR CORP       1997-05-31  1004.0  31/05/1997   
4  1750  AAR CORP       1998-05-31  1004.0  31/05/1998   

                                        filename  can_label qualified  label  \
0  /1994/1750_10K_1994_0000912057-94-002818.json       True       Yes  False   
1  /1995/1750_10K_1995_0000912057-95-006316.json       True       Yes  False   
2  /1996/1750_10K_1996_0000912057-96-018355.json       True       Yes  False   
3  /1997/1750_10K_1997_0000912057-97-028915.json       True       Yes  False   
4  /1998/1750_10K_1998_0001047469-98-032283.json       True       Yes  False   

  bankruptcy_prediction_split bankruptcy_date_1 bankruptcy_date_2  \
0                       train               NaN               NaN   
1                       train   

In [8]:
companies_with_not_found_tickers = pd.read_csv('ECL_companies_with_not_found_tickers_2024-04-12_19-10-45.csv', index_col=0)

In [9]:
def pad_cik(cik):
    cik = str(cik)
    while len(cik) < 10:
        cik = '0' + cik
    return cik

In [ ]:
from tqdm import tqdm
import requests

tqdm.pandas()

not_found_tickers_count = 0
exceptions_count = 0

def process_row(row):
    global not_found_tickers_count
    global exceptions_count
    
    cik = row['cik']
    company = row['company']
    url = f'https://www.sec.gov/edgar/browse/?CIK={cik}&owner=exclude'
    
    try:
        response = requests.get(url)
        response_json = response.json()

        indexes = []
        cnt = 0
        for exchange in response_json['exchanges']:
            if exchange in ('NYSE', 'Nasdaq'):
                indexes.append(cnt)
                cnt += 1
                
        tickers = response_json['tickers']
        
        for index in indexes:
            ticker = tickers[index]
            exchange = response_json['exchanges'][index]
            
            partially_filled_dataset.loc[partially_filled_dataset['cik'] == cik, 'ticker'] = ticker
        
            if exchange == 'Nasdaq':
                exchange = 'NAS'
            partially_filled_dataset.loc[partially_filled_dataset['cik'] == cik, 'exchange'] = exchange


    except Exception as e:
        # print(f"Error for company: {company}, cik: {cik} Error: {e}")
        exceptions_count += 1
        return {'cik': cik, 'company': company}

result_df  = unique_companies[:400].progress_apply(process_row, axis=1)
result_df = result_df.dropna().reset_index(drop=True)
companies_with_not_found_tickers_new = pd.DataFrame(result_df.tolist())

driver.close()